In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

In [ ]:
years = list(range(2024, 2021, -1))
all_matches = []
standings_url = "https://www.basketball-reference.com/leagues/NBA_2024.html"
seasons = []

In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table#per_game-team')[0]
    
    links = standings_table.find_all('a')
    links = [link.get("href") for link in links]
    team_urls = [f"https://www.basketball-reference.com{l}" for l in links]

    previous_season = soup.select("a.button2.prev")[0].get("href")
    standings_url = f"https://www.basketball-reference.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-2]
        data = requests.get(team_url)
        
        gameLink = BeautifulSoup(data.text)
        links = gameLink.select('div#bottom_nav_container.section_content ul li')
        
        for l in links:
            link = l.find('a')
            if link and "Game Log" in link.get_text():
                gameLink = link.get("href")

        gameLink = f"https://www.basketball-reference.com{gameLink}"
        data = requests.get(gameLink)
        gameLog = StringIO(data.text)
        matches = pd.read_html(gameLog, match="Regular Season")[0]
        matches["Season"] = year
        matches["team"] = team_name

        matches.columns = matches.columns.droplevel()
        new_columns = matches.columns.tolist()
        new_columns[41] = 'season'
        new_columns[42] = 'team'
        for i in range(25, 41):
            if(not('Opp_' in matches.columns[i])):
                 new_columns[i] = 'Opp_' + new_columns[i]

        matches.columns = new_columns

        all_matches.append(matches)
        time.sleep(3)

In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("nba_games")